In [ ]:
#Create new virtual environment
#pip install pandas
#pip install pycaret

import pandas as pd
import pycaret
from pycaret.classification import *

set_config('seed', 999)

data = pd.read_csv('train_pycaret.csv')

s = setup(data=data, target="Transported",  fold_shuffle=True, session_id=999, silent=True)
compare_models()


***




# Set SEED for reproducibility
SEED = 37

# Create the DMatrix
import xgboost as xgb
dmatrix = xgb.DMatrix(data=X_train, label=y_train)

# specify parameters via map
param = {'booster': 'dart',
         'max_depth': 5, 'learning_rate': 0.1,
         'objective': 'binary:logistic',
         'sample_type': 'uniform',
         'normalize_type': 'tree',
         'rate_drop': 0.1,
         'skip_drop': 0.5}
num_round = 50

# Perform cross-validation: cv_results
cv_results = xgb.cv(dtrain=dmatrix, params=param, 
                  nfold=10, num_boost_round=25, 
                  metrics="error", as_pandas=True, seed=123)

# Print the accuracy
print(((1-cv_results["test-error-mean"]).iloc[-1]))

****


# Instantiate the regressor
gbm = xgb.XGBClassifier()

# Cross-valindation
from sklearn.model_selection import GridSearchCV, KFold
kf = KFold(n_splits=6, shuffle=True, random_state=SEED)

# Perform grid search
param_grid = {
    'n_estimators': [10],
    "objective": ["binary:logistic"],
    'max_depth': [10, 15, 20]
}
grid = GridSearchCV(param_grid=param_grid, estimator=gbm, scoring='accuracy', cv=3, verbose=2)

# Fit grid_mse to the data
grid.fit(X_train, y_train)


print("Best parameters found: ", grid.best_params_)
print("Best accuracy: ", grid.best_score_)


###

# Create the parameter dictionary: params
params = {"objective":"binary:logistic", "max_depth":15}

# Perform cross-validation: cv_results
cv_results = xgb.cv(dtrain=dmatrix, params=params, 
                  nfold=10, num_boost_round=15, 
                  metrics="error", as_pandas=True, seed=123)

###

# Create the parameter grid
gbm_param_grid = {
    'clf__learning_rate': np.arange(0.05, 1.00, 0.05),
    'clf__max_depth': range(3, 10, 1),
    'clf__n_estimators': range(50, 200, 50)
}

# Perform RandomizedSearchCV
randomized_roc_auc = RandomizedSearchCV(pipeline, param_distributions=gbm_param_grid, n_iter=2, scoring='roc_auc', cv=2, verbose=1)

# Fit the estimator
randomized_roc_auc.fit(X, y)

# Compute metrics
print(randomized_roc_auc.best_score_)
print(randomized_roc_auc.best_estimator_)
